In [1]:
!pip install minio requests boto3



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import requests
import json
import boto3
from botocore.exceptions import NoCredentialsError
from datetime import datetime
import os
import sys

In [6]:
try:
    import boto3
    import requests
except ImportError:
    print("Dependencies not found. Please run: pip install minio requests boto3")
    sys.exit(1)

In [20]:
INFERENCE_ENDPOINT = "https://granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MODEL_API_URL = f"{INFERENCE_ENDPOINT}/v1/completions"
MODEL_NAME = "granite"

# MinIO Storage Settings
MINIO_ENDPOINT = "minio-api-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MINIO_ACCESS_KEY = "minio"
MINIO_SECRET_KEY = "minio123"
MINIO_BUCKET = "logs"
LOG_FILE_NAME = "httpd_down.txt"

In [21]:

def setup_minio_log_file():
    """
    Ensures the log file exists in MinIO with the correct content.
    This function creates/overwrites the log file to prevent issues
    from failed manual uploads.
    """
    print(f"Ensuring '{LOG_FILE_NAME}' exists in MinIO bucket '{MINIO_BUCKET}'...")
    
    # Log content that should be in the file
    log_data = """
# === Scenario: httpd service fails to start due to a configuration error ===
# Hostname: ex2.rhel.instance
# Timestamp: June 12, 2025
[admin@ex2 ~]$ sudo systemctl restart httpd
Job for httpd.service failed because the control process exited with error code.
[admin@ex2 ~]$ sudo journalctl -u httpd --since "5 minutes ago"
Jun 12 18:07:34 ex2.rhel.instance systemd[1]: Starting The Apache HTTP Server...
Jun 12 18:07:34 ex2.rhel.instance httpd[4521]: AH00526: Syntax error on line 96 of /etc/httpd/conf/httpd.conf:
Jun 12 18:07:34 ex2.rhel.instance httpd[4521]: Invalid command 'Listenport', perhaps misspelled or defined by a module not included in the server configuration
Jun 12 18:07:34 ex2.rhel.instance systemd[1]: httpd.service: Main process exited, code=exited, status=1/FAILURE
"""
    try:
        s3_client = boto3.client(
            's3',
            endpoint_url=f"http://{MINIO_ENDPOINT}",
            aws_access_key_id=MINIO_ACCESS_KEY,
            aws_secret_access_key=MINIO_SECRET_KEY
        )
        # put_object will create the file or overwrite it if it exists.
        s3_client.put_object(Body=log_data.encode('utf-8'), Bucket=MINIO_BUCKET, Key=LOG_FILE_NAME)
        print(f"✅ Successfully created/updated '{LOG_FILE_NAME}' in MinIO.")
        return True
    except NoCredentialsError:
        print("❌ Error: Credentials not available. Check your MinIO access/secret keys.")
    except Exception as e:
        print(f"❌ An error occurred while setting up the log file in MinIO: {e}")
    return False

def read_log_from_minio(bucket_name, object_name):
    """Reads a log file's content from a MinIO bucket."""
    print(f" Attempting to read '{object_name}' from bucket '{bucket_name}'...")
    try:
        s3_client = boto3.client(
            's3',
            endpoint_url=f"http://{MINIO_ENDPOINT}",
            aws_access_key_id=MINIO_ACCESS_KEY,
            aws_secret_access_key=MINIO_SECRET_KEY
        )
        response = s3_client.get_object(Bucket=bucket_name, Key=object_name)
        content = response['Body'].read().decode('utf-8')
        print(f"✅ Successfully read log content from MinIO.")
        return content
    except s3_client.exceptions.NoSuchKey:
        print(f"❌ Error: The object '{object_name}' was not found in bucket '{bucket_name}'.")
    except Exception as e:
        print(f"❌ An error occurred while reading from MinIO: {e}")
    return None

def query_model_for_report(logs):
    """Queries the Granite model with a specific prompt to generate an incident report."""
    prompt = f"""
Generate a concise, structured AIOps incident report based on the following logs. The report must be in Markdown format and include:
1. Key metadata: INCIDENT ID, DETECTED, SEVERITY, STATUS, AFFECTED SERVICE, AFFECTED HOST, SUMMARY, and KEY ERROR LOG.
2. A ROOT CAUSE section explaining the 'why'.
3. A REMEDIATION PLAYBOOK section with exact, numbered shell commands to fix the issue.
4. A VALIDATION section with a command to confirm the fix.

--- LOGS START ---
{logs}
--- LOGS END ---
"""
    headers = {"Content-Type": "application/json"}
    # **EDIT**: Added 'max_tokens' to ensure the model generates a complete report.
    payload = {
        "model": MODEL_NAME, 
        "prompt": prompt,
        "max_tokens": 512 
    }
    
    print("\n querying Granite model for analysis...")
    try:
        response = requests.post(MODEL_API_URL, headers=headers, json=payload, verify=False)
        response.raise_for_status() # Raise an exception for bad status codes
        
        result = response.json()
        if 'choices' in result and result['choices']:
            report_text = result['choices'][0].get('text', 'Error: Could not extract report from model response.')
            print("✅ Analysis complete.")
            return report_text
        else:
            return f"### Error\nUnexpected response format from model: `{json.dumps(result)}`"
    except requests.exceptions.RequestException as e:
        return f"### Error\nCould not connect to model at `{MODEL_API_URL}`. Details: {e}"
    except Exception as e:
        return f"### Error\nAn unexpected exception occurred: {e}"


In [ ]:

def main():
    """Main function to run the AIOps workflow."""
    # Step 1: Ensure the log file exists in MinIO with the correct content.
    if not setup_minio_log_file():
        print("Aborting script due to failure in setting up log file.")
        return

    # Step 2: Read the specified log file from MinIO
    log_content = read_log_from_minio(MINIO_BUCKET, LOG_FILE_NAME)
    
    # Step 3: Check if logs were read successfully and are not empty
    if log_content and log_content.strip():
        print("\n--- Content of httpd_down.txt ---")
        print(log_content)
        print("---------------------------------\n")

        incident_report = query_model_for_report(log_content)
        
        print("\n" + "="*50)
        print("     AIOps Incident Remediation Report")
        print("="*50 + "\n")
        print(incident_report)
    elif log_content is not None:
        print("\n❌ Analysis skipped: The log file '{LOG_FILE_NAME}' is empty.")
    else:
        print(f"\n❌ Analysis skipped: Failed to read '{LOG_FILE_NAME}' from MinIO.")

if __name__ == "__main__":
    main()

Ensuring 'httpd_down.txt' exists in MinIO bucket 'logs'...
✅ Successfully created/updated 'httpd_down.txt' in MinIO.
 Attempting to read 'httpd_down.txt' from bucket 'logs'...
✅ Successfully read log content from MinIO.

--- Content of httpd_down.txt ---

# === Scenario: httpd service fails to start due to a configuration error ===
# Hostname: ex2.rhel.instance
# Timestamp: June 12, 2025
[admin@ex2 ~]$ sudo systemctl restart httpd
Job for httpd.service failed because the control process exited with error code.
[admin@ex2 ~]$ sudo journalctl -u httpd --since "5 minutes ago"
Jun 12 18:07:34 ex2.rhel.instance systemd[1]: Starting The Apache HTTP Server...
Jun 12 18:07:34 ex2.rhel.instance httpd[4521]: AH00526: Syntax error on line 96 of /etc/httpd/conf/httpd.conf:
Jun 12 18:07:34 ex2.rhel.instance httpd[4521]: Invalid command 'Listenport', perhaps misspelled or defined by a module not included in the server configuration
Jun 12 18:07:34 ex2.rhel.instance systemd[1]: httpd.service: Main pr

/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
